# PineconeRetrievalChain 테스트 노트북

In [1]:
import os
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

if PINECONE_API_KEY:
    print("Pinecone API Key가 제대로 로드되었습니다.")
else:
    print("Pinecone API Key가 로드되지 않았습니다. .env 파일을 확인하세요.")

Pinecone API Key가 제대로 로드되었습니다.


In [2]:
# Test 1: Pinecone 인덱스 생성 및 초기화
import pinecone_ as pinecone

# 환경 변수 로드 (dotenv 파일 설정이 되어 있어야 합니다)
from dotenv import load_dotenv
load_dotenv()

# Pinecone API Key 및 인덱스 이름
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME", "liberty-rag-json")

# Pinecone 인덱스 초기화
retrieval_chain = pinecone.PineconeRetrievalChain(index_name=PINECONE_INDEX_NAME)
print(f"Pinecone 인덱스가 성공적으로 초기화되었습니다: {retrieval_chain.pinecone_params}")

인덱스 'liberty-rag-json'가 이미 존재합니다.
Pinecone 인덱스가 성공적으로 초기화되었습니다: {'index': <pinecone.data.index.Index object at 0x163db8fd0>, 'namespace': 'liberty-rag-json-namespace-01'}


### Pinecone Index 설정

In [3]:
from pinecone import Index, init
import os
from dotenv import load_dotenv
load_dotenv()
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_NAME=os.getenv("PINECONE_INDEX_NAME", "liberty-ai-index")
def create_index(api_key: str=PINECONE_API_KEY, index_name: str=PINECONE_INDEX_NAME, dimension: int = 768, metric: str = "dotproduct", host="https://liberty-rag-json-hwsbh8f.svc.aped-4627-b74a.pinecone.io"):
    """Pinecone 인덱스를 생성하고 반환합니다."""
    # Pinecone 클라이언트를 API 키로 초기화
    #init(api_key=api_key,environment=PINECONE_ENVIRONMENT)
    
    try:
            # 이미 존재하는 인덱스를 가져옴
        index = Index(index_name, host=host)
        print(f"인덱스 '{index_name}'가 이미 존재합니다.")
    except Exception as e:
        # 인덱스가 존재하지 않으면 새로 생성
        print(f"인덱스 '{index_name}'가 존재하지 않아서 새로 생성합니다.")
        from pinecone import create_index
        create_index(
                name=index_name,
                dimension=dimension,  # 임베딩 차원수 (모델에 맞게 설정)
                metric=metric
            )
        index = Index(index_name, host=host)
        print(f"새로운 인덱스 '{index_name}' 생성 완료")
        
        return index

In [4]:

# #"""Pinecone 인덱스를 초기화하거나 존재하지 않으면 생성합니다."""
# host = "https://liberty-index-hwsbh8f.svc.aped-4627-b74a.pinecone.io"
# index = create_index(api_key=PINECONE_API_KEY, host=host, index_name=PINECONE_INDEX_NAME)


In [5]:
from pinecone import Pinecone

# Pinecone 클라이언트 초기화
host = "https://liberty-rag-json-hwsbh8f.svc.aped-4627-b74a.pinecone.io"
pc = Pinecone(api_key=PINECONE_API_KEY)

# 인덱스 접근
index = pc.Index(host=host)  # describe_index()로 host 확인 가능
pinecone_params = {"index": index, "namespace": PINECONE_INDEX_NAME + "-namespace-01"}

# 인덱스와 pinecone_params 테스트
print(f"인덱스 이름: {index.describe_index_stats()}")
print(f"네임스페이스: {pinecone_params['namespace']}")


인덱스 이름: {'dimension': 4096,
 'index_fullness': 0.0,
 'namespaces': {'liberty-rag-json-namespace-01': {'vector_count': 57104}},
 'total_vector_count': 57104}
네임스페이스: liberty-rag-json-namespace-01


### 문서 업로드(json loading, split_docs)

In [6]:
# Test 2: 문서 로드 및 전처리 테스트
import pinecone_
import os

# 데이터 경로 설정
data_dir = "data/154.의료, 법률 전문 서적 말뭉치/01-1.정식개방데이터/Training/02.라벨링데이터/Training_legal.json"  # data 폴더에 PDF 파일들을 저장해 주세요
import json
split_docs=[]
with open(data_dir, 'r', encoding='utf-8') as f:
    try:
        json_data = json.load(f)
                    
        # data 배열에서 문서 추출 (처음 10개만)
        if isinstance(json_data, dict) and 'data' in json_data:
            for item in json_data['data']:  # 처음 10개의 문서만 로드
                if isinstance(item, dict):
                    # Document 객체 생성
                    from langchain.schema import Document
                    doc = Document(
                        page_content=item.get('text', ''),
                        metadata={
                            'book_id': item.get('book_id'),
                            'category': item.get('category'),
                            'popularity': item.get('popularity'),
                            'keyword': item.get('keyword', []),
                            'word_segment': item.get('word_segment', []),
                            'publication_ymd': item.get('publication_ymd')
                        }
                    )
                    split_docs.append(doc)
                else:
                    print("JSON 데이터가 예상된 형식이 아닙니다.")
                    print("데이터 구조:", json_data.keys() if isinstance(json_data, dict) else type(json_data))
            
            print(f"전체 {len(json_data['data'])}개 중 {len(split_docs)}개의 문서를 로드했습니다.")
                        
    except json.JSONDecodeError as e:
        print(f"JSON 파일 파싱 중 오류가 발생했습니다: {e}")
print(split_docs[0].metadata)
print(split_docs[0].page_content)
print(split_docs[0].metadata['book_id'])
print(split_docs[0].metadata['category'])
print(split_docs[0].metadata['popularity'])
print(split_docs[0].metadata['keyword'])
print(split_docs[0].metadata['word_segment'])
print(split_docs[0].metadata['publication_ymd'])

전체 63704개 중 63704개의 문서를 로드했습니다.
{'book_id': 'LJU000001', 'category': '환경/교통법', 'popularity': 3, 'keyword': ['개발제한구역의 지정 및 관리에 관한 특별조치법', '시정명령', '개발제한구역법', '원심판결을 파기', '도시계획법'], 'word_segment': 832, 'publication_ymd': '20131212'}
이행강제금부과처분취소 (대법원 2013. 12. 12. 선고 2012두20397 판결) 【출전】 판례공보 제434호, 2014년 1월 15일 193페이지 【판시사항】 [1] 개발제한구역의 지정 및 관리에 관한 특별조치법상 이행강제금을 부과·징수할 때마다 그에 앞서 시정명령 절차를 다시 거쳐야 하는지 여부(소극) [2] 개발제한구역의 지정 및 관리에 관한 특별조치법에 의한 이행강제금 부과의 근거가 되는 시정명령이 이루어져야 하는 시기(=법률 시행일인 2010. 2. 7. 이후) 【판결요지】 [1] 개발제한구역의 지정 및 관리에 관한 특별조치법 제30조 제1항, 제30조의2 제1항 및 제2항의 규정에 의하면 시정명령을 받은 후 그 시정명령의 이행을 하지 아니한 자에 대하여 이행강제금을 부과할 수 있고, 이행강제금을 부과하기 전에 상당한 기간을 정하여 그 기한까지 이행되지 아니할 때에 이행강제금을 부과·징수한다는 뜻을 문서로 계고하여야 하므로, 이행강제금의 부과·징수를 위한 계고는 시정명령을 불이행한 경우에 취할 수 있는 절차라 할 것이고, 따라서 이행강제금을 부과·징수할 때마다 그에 앞서 시정명령 절차를 다시 거쳐야 할 필요는 없다. [2] 구 개발제한구역의 지정 및 관리에 관한 특별조치법(2009. 2. 6. 법률 제9436호로 개정되기 전의 것)이나 개발제한구역의 지정 및 관리에 관한 특별조치법(이하 '개발제한구역법'이라 한다)이 제정·시행되기 전의 구 도시계획법(2000. 1. 28. 법률 제6243호로 전부 개정되기 전의 것)은 개발제한구역에서 행위제한

### 문서 전처리

In [15]:
from typing import List, Any
from tqdm import tqdm
def preprocess_documents(
    split_docs: List[Any], 
    metadata_keys: List[str] = ["book_id", "category", "popularity", "keyword", "word_segment", "publication_ymd"], 
    min_length: int = 5
):
    """문서를 전처리하고 내용과 메타데이터를 반환합니다."""
    contents = []
    metadatas = {key: [] for key in metadata_keys}
    
    for doc in tqdm(split_docs, desc="문서 전처리 중"):
        content = doc.page_content.strip()
        if content and len(content) >= min_length:
            # 컨텍스트 길이 제한 (Pinecone 권장사항)
            contents.append(content[:4000])  
            
            # 메타데이터 추출
            for k in metadata_keys:
                value = doc.metadata.get(k)
                metadatas[k].append(value)
                
    print(f"전체 {len(split_docs)}개 중 {len(contents)}개의 문서가 전처리되었습니다.")
    return contents, metadatas

In [16]:
contents, metadatas = preprocess_documents(split_docs)

문서 전처리 중: 100%|██████████| 63704/63704 [00:00<00:00, 182826.35it/s]

전체 63704개 중 63704개의 문서가 전처리되었습니다.


### Embedder

In [17]:

from langchain_teddynote.korean import stopwords

# 한글 불용어 사전 불러오기 (불용어 사전 출처: https://www.ranks.nl/stopwords/korean)
stopword = stopwords()
#stopword

In [26]:
from langchain_teddynote.community.pinecone import (
    create_sparse_encoder,
    fit_sparse_encoder,
)

# 한글 불용어 사전 + Kiwi 형태소 분석기를 사용합니다.
sparse_encoder = create_sparse_encoder(stopwords(), mode="kiwi")

In [27]:
# Sparse Encoder 를 사용하여 contents 를 학습
saved_path = fit_sparse_encoder(
    sparse_encoder=sparse_encoder, contents=contents, save_path="./sparse_encoder.pkl"
)

  0%|          | 0/63704 [00:00<?, ?it/s]

[fit_sparse_encoder]
Saved Sparse Encoder to: ./sparse_encoder.pkl


In [11]:
from langchain_teddynote.community.pinecone import load_sparse_encoder

# 추후에 학습된 sparse encoder 를 불러올 때 사용합니다.
sparse_encoder = load_sparse_encoder("./sparse_encoder.pkl")

[load_sparse_encoder]
Loaded Sparse Encoder from: ./sparse_encoder.pkl


In [12]:
import pickle
from langchain_upstage import UpstageEmbeddings
from langchain_teddynote.community.kiwi_tokenizer import KiwiBM25Tokenizer
passage_embeddings = UpstageEmbeddings(model="solar-embedding-1-large-query")

    

In [13]:
# Test 3: Sparse Encoder 학습 및 로드 
import os
import time
import pickle
import secrets
import itertools
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm
import glob
from dotenv import load_dotenv
from pinecone import Index, init, Pinecone
from langchain_upstage import UpstageEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PDFPlumberLoader
from typing import List, Dict, Any
from langchain_teddynote.community.kiwi_tokenizer import KiwiBM25Tokenizer
def generate_hash() -> str:
    """24자리 무작위 hex 값을 생성하고 6자리씩 나누어 '-'로 연결합니다."""
    random_hex = secrets.token_hex(12)
    return "-".join(random_hex[i: i + 6] for i in range(0, 24, 6))

from pinecone import Index, init
import itertools
def chunks(iterable, size):
    it = iter(iterable)
    chunk = list(itertools.islice(it, size))
    while chunk:
        yield chunk
        chunk = list(itertools.islice(it, size))

# def upsert_documents_parallel(contents, metadatas, sparse_encoder, pinecone_params, embedder=UpstageEmbeddings(model="solar-embedding-1-large-query"), batch_size=100, max_workers=30):
#     # 문자열을 Document 객체로 변환
from langchain.schema import Document
keys = list(metadatas.keys())
embedder=passage_embeddings
    
    # documents = [
    #     Document(
    #         page_content=text,
    #         metadata={
    #             'book_id': metadatas['book_id'][i],
    #             'category': metadatas['category'][i], 
    #             'popularity': metadatas['popularity'][i],
    #             'keyword': metadatas['keyword'][i],
    #             'word_segment': metadatas['word_segment'][i],
    #             'publication_ymd': metadatas['publication_ymd'][i]
    #         }
    #     ) for i, text in enumerate(batch)
    # ]

def process_batch(batch, contents, metadatas, sparse_encoder, passage_embeddings, pinecone_params):
    """
    배치 단위로 문서를 처리하여 Pinecone에 업로드합니다.
    
    Args:
        batch: 현재 처리할 문서들의 인덱스 리스트
        contents: 전체 문서 내용 리스트
        metadatas: 전체 문서의 메타데이터 딕셔너리
        sparse_encoder: 희소 임베딩을 생성하는 인코더
        passage_embeddings: 밀집 임베딩을 생성하는 인코더
        pinecone_params: Pinecone 관련 설정
    """
    # 현재 배치에 해당하는 문서와 메타데이터 추출
    context_batch = [contents[i] for i in batch]
    metadata_keys = ['book_id', 'category', 'popularity', 'keyword', 'word_segment', 'publication_ymd']
    
    batch_result = [
        {
            "context": context[:1000],  # 컨텍스트 길이 제한
            **{key: metadatas[key][i] for key in metadata_keys}
        } for i, context in enumerate(context_batch)
    ]

    ids = [generate_hash() for _ in range(len(batch))]
    dense_embeds = passage_embeddings.embed_documents(context_batch)
    sparse_embeds = sparse_encoder.encode_documents(context_batch)
    print(f"Processing batch: {batch}")
    vectors = [
        {
            "id": id_,
            "values": dense_embed,
            "sparse_values": sparse_embed,
            "metadata": metadata
        }
        for id_, dense_embed, sparse_embed, metadata in zip(
            ids, dense_embeds, sparse_embeds, batch_result
        )
    ]

    try:
        return index.upsert(
            vectors=vectors,
            namespace=pinecone_params["namespace"],
            async_req=False
        )
    except Exception as e:
        print(f"Upsert 중 오류 발생: {e}")
        return None

In [35]:
batch_size=100
max_workers=30
# 57,005번째 문서부터 처리
start_idx = 57005
contents_subset = contents[start_idx:]
metadata_subset = {key: values[start_idx:] for key, values in metadatas.items()}

batches = list(chunks(range(len(contents_subset)), batch_size))
print(batches)
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [
        executor.submit(
            process_batch, 
            batch,
            contents_subset,
            metadata_subset,
            sparse_encoder,
            passage_embeddings,
            pinecone_params
        ) for batch in batches
    ]
    print(futures)
    results = []
    
    # tqdm 설정 추가
    total_docs = len(contents_subset)
    pbar = tqdm(total=total_docs, desc="문서 Upsert 중")
    start_time = time.time()
    
    processed_docs = 0
    for idx, future in enumerate(as_completed(futures)):
        result = future.result()
        if result:
            results.append(result)
            
        # 진행률과 예상 시간 계산
        elapsed_time = time.time() - start_time
        processed_docs += len(batches[idx])
        progress = processed_docs / total_docs
        estimated_total_time = elapsed_time / progress if progress > 0 else 0
        remaining_time = estimated_total_time - elapsed_time
        
        # 진행 상태 업데이트
        pbar.set_postfix({
            'progress': f'{processed_docs}/{total_docs}',
            'remaining': f'{remaining_time:.1f}s'
        })
        pbar.update(len(batches[idx]))
    
    pbar.close()
    total_upserted = sum(result.upserted_count for result in results if result)
    print(f"총 {total_upserted}개의 Vector가 Upsert 되었습니다.")
    print(f"{pinecone_params['index'].describe_index_stats()}")

[]
[]


문서 Upsert 중: 0it [00:00, ?it/s]

총 0개의 Vector가 Upsert 되었습니다.
{'dimension': 4096,
 'index_fullness': 0.0,
 'namespaces': {'liberty-rag-json-namespace-01': {'vector_count': 57004}},
 'total_vector_count': 57004}


In [19]:
# 57,005번째 문서부터 처리하기 위한 데이터 슬라이싱
start_idx = min(63004, len(contents))
contents_subset = contents[start_idx:]
metadata_subset = {key: values[start_idx:] for key, values in metadatas.items()}
print(f"Contents subset length: {len(contents_subset)}")
print(f"Metadata subset length: {len(metadata_subset)}")
batch_size = 100
max_workers = 30

# 배치 생성
batches = list(chunks(range(len(contents_subset)), batch_size))
print(f"Number of batches: {len(batches)}")
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [
        executor.submit(
            process_batch, 
            batch,
            contents_subset,  # 슬라이싱된 contents 사용
            metadata_subset,  # 슬라이싱된 metadata 사용
            sparse_encoder,
            passage_embeddings,
            pinecone_params
        ) for batch in batches
    ]
    print(f"Number of futures: {len(futures)}")
    # 진행 상황 모니터링
    total_docs = len(contents_subset)
    pbar = tqdm(total=total_docs, desc="문서 Upsert 중")
    start_time = time.time()
    
    results = []
    processed_docs = 0
    
    for idx, future in enumerate(as_completed(futures)):
        result = future.result()
        if result:
            results.append(result)
            
        processed_docs += len(batches[idx])
        elapsed_time = time.time() - start_time
        progress = processed_docs / total_docs
        estimated_total_time = elapsed_time / progress if progress > 0 else 0
        remaining_time = estimated_total_time - elapsed_time
           
        pbar.set_postfix({
            'progress': f'{processed_docs}/{total_docs}',
            'remaining': f'{remaining_time:.1f}s'
        })
        pbar.update(len(batches[idx]))
    
    pbar.close()

Contents subset length: 700
Metadata subset length: 6
Number of batches: 7
Number of futures: 7


Processing batch: [400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499]
Processing batch: [600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 69

Processing batch: [300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399]


Processing batch: [100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199]


문서 Upsert 중:  14%|█▍        | 100/700 [01:03<06:06,  1.64it/s, progress=200/700, remaining=156.2s]

Processing batch: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
Processing batch: [200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299]



문서 Upsert 중:  71%|███████▏  | 500/700 [01:18<00:18, 10.79it/s, progress=500/700, remaining=31.1s]

Processing batch: [500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599]


문서 Upsert 중: 100%|██████████| 700/700 [01:20<00:00,  8.67it/s, progress=700/700, remaining=0.0s]


In [ ]:
# 기존 네임스페이스삭제
pinecone_params['index'].delete(namespace=pinecone_params['namespace'])
print(f"네임스페이스 '{pinecone_params['namespace']}'가 성공적으로 삭제되었습니다.")